data_input_excel_to_db 버전에서는 db 갯수 조회와 add하는것을 적용시켰다.

data_input_excel_to_db_02 버전에서는 기존 db 갯수 조회 및 삭제, 이후 add 시키는것을 적용시킨다.

data_input_excel_to_db_03 버전에서는 기존 데이터 입력 방식의 개선을 진행한다.(0719추가 내용 확인)

ㄴ> 기존 기능 내용에 대해 풀어서 표시된것은 data_input_excel_to_db_02_for test <-- 를 참고하자

▼▼▼ 이하 예제 시작(question_list에 적용하는것 => ss_project 적용)

In [ ]:
#데이터 삭제 구현 완료 , 중간 하드코딩 건너띄고, 연속 데이터 입력 적용 확인

In [1]:
from pybo.models import Question, User
from pybo import db

from flask import Blueprint, url_for, render_template, flash, request, session

from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import redirect

from flask import Flask
from flask_migrate import Migrate
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import MetaData
import config

import pybo

from openpyxl import load_workbook
from datetime import datetime
from pybo import db, create_app

from pybo import create_app
pybo = create_app()
pybo.app_context().push()

In [2]:
print(Question.query.count())

20


In [3]:
print(User.query.count())

9


In [4]:
app = Flask(__name__)
app.config.from_object(config)
    #app.config 환경 변수로 부르기 위해 app.config.from_object(config)코드를 추가함

In [ ]:
#위에 까지가 기본 세팅 완료

In [ ]:
#user 관련 확인용_220623

In [8]:
user_list_00=User.query.all()

In [9]:
user_list_00

[<User 2>, <User 4>, <User 6>, <User 7>, <User 8>]

In [10]:
a_user = user_list_00[0]

In [11]:
a_user

<User 2>

In [12]:
a_user.username

'관리자02'

In [ ]:
# 데이터 삭제시 각 내용을 리스트화 해서 id값이 다르게 점프해도 삭제할수 있도록

In [30]:
query_list_00=Question.query.all()

In [32]:
len(query_list_00)

899

In [33]:
query_list_00[0]

<Question 1>

In [20]:
query_test=Question.query.get(1000)
print(query_test)

<Question 1000>


In [ ]:
# 데이터 삭제시 각 내용을 리스트화 해서 id값이 다르게 점프해도 삭제할수 있도록

In [ ]:
# 개선 내용 테스트 해본다.

In [96]:
### 데이터 삭제 기능 개선_리스트화 적용
data_count = 0
query_list_00=Question.query.all()

with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ', Question.query.count())
    query_count_num = Question.query.count()
    
    for query_del_num in range(query_count_num):
        query_del_test = query_list_00[query_del_num]
        db.session.delete(query_del_test)
        db.session.commit()
        db.session.close()
        data_count = data_count+1
    
print("최종 작업 이후 Question모델 데이터 개수 : ", Question.query.count())
print("작업 진행 데이터 개수 : ", data_count)

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) database is locked
[SQL: DELETE FROM question WHERE question.id = ?]
[parameters: (1,)]
(Background on this error at: https://sqlalche.me/e/14/e3q8) (Background on this error at: https://sqlalche.me/e/14/7s2a)

ㄴ> 기존버전의 id값 기준 삭제하는 부분을 제거 하였다.

In [ ]:
### 입력 부분 기능 시작

이제 연속해서 하는걸 해보자

▼▼▼▼ 데이터 입력부 시작

In [5]:
from openpyxl import load_workbook

In [6]:
wb = load_workbook(filename='db_test.xlsx')
ws = wb.active
ws

<Worksheet "Sheet1">

In [7]:
print(Question.query.count())

20


In [19]:
a = 0
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        try:
            #user = User(username='shin12352', password='shin_123452', email='shin52@naver.com')
            #user = User(username=ws.cell(row,1).value, password=ws.cell(row,2).value, email=ws.cell(row,3).value)
            question = Question(our_ref=ws.cell(row,1).value, pay_date=ws.cell(row,2).value, req_name=ws.cell(row,3).value, cus_name=ws.cell(row,4).value, date=ws.cell(row,5).value, number=ws.cell(row,6).value, q_date=ws.cell(row,7).value, q_d_date=ws.cell(row,8).value, d_d_date=ws.cell(row,9).value, d_d_num=ws.cell(row,10).value, p_name=ws.cell(row,11).value, charge_01=ws.cell(row,12).value, charge_02=ws.cell(row,13).value, p_charge=ws.cell(row,14).value, create_date=datetime.now())
            #user = User(username=ws.cell(row,1).value, password=ws.cell(row,2).value, email=ws.cell(row,3).value)
            #print(question)
            db.session.add(question)
            db.session.commit()
            #print('사용자 정보를 생성 하였습니다.')
        except:
            db.session.rollback()
            #print('입력에 에러가 발생 하였습니다.')
            pass
        finally:
            db.session.close()
        #print(a," 번째 작업을 종료 합니다.")
        a = a+1
        #print('='*10)

print("최종 작업 이후 Question모델 데이터 개수 : ", Question.query.count())
print("작업 진행 데이터 개수 : ", a)

최초 시작시 Question모델 데이터 개수 :  0
최종 작업 이후 Question모델 데이터 개수 :  20
작업 진행 데이터 개수 :  20


▲▲▲ 이상 예제 종료(question_list에 적용하는것 => ss_project 적용)

▼▼▼▼ 이하 신규 데이터 입력부 시작_0719

In [75]:
from openpyxl import load_workbook

In [10]:
wb = load_workbook(filename='db_test.xlsx')
ws = wb.active
ws

<Worksheet "Sheet1">

In [11]:
print(Question.query.count())

20


In [8]:
Question.query.all()

[<Question 1>,
 <Question 2>,
 <Question 3>,
 <Question 4>,
 <Question 5>,
 <Question 6>,
 <Question 7>,
 <Question 8>,
 <Question 9>,
 <Question 10>,
 <Question 11>,
 <Question 12>,
 <Question 13>,
 <Question 14>,
 <Question 15>,
 <Question 16>,
 <Question 17>,
 <Question 18>,
 <Question 19>,
 <Question 20>]

In [31]:
q = Question.query.get(2)

In [32]:
q

<Question 2>

In [33]:
q.our_ref

'D211115'

In [44]:
Question.query.filter(Question.our_ref.ilike('D211115')).all()

[<Question 2>]

In [45]:
test_b=Question.query.filter(Question.our_ref.ilike('D211115')).all()

In [46]:
test_b

[<Question 2>]

In [37]:
a = 0
same_count = 0
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        print_ques=ws.cell(row,1).value
        #print_ques=ws.cell(row,3) <-- 위치값을 표시한다.
        print(print_ques)
        
        

최초 시작시 Question모델 데이터 개수 :  20
D211114
D211115
D211116
P212398
P212400
P212412
P211279-RE
P212405
P212418
P212430-DIV1
P212430-DIV2
P212419
P212420
P212421
P211363-PRO2
P212441
P209250-DIV1
P212439-PRO1
P212439-PRO2
P212434-PRO1


이 다음은 엑셀 파일에서 읽은값과 기존 question모델의 db 값과 같은지 확인해보자.

In [15]:
a = 0
same_count = 0
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        # 엑셀 파일 내에 하나씩 읽어 오는 값을~~
        # 기존의 데이터와 비교해야 겠다.
        print_ques=ws.cell(row,1).value #<---- 엑셀 sheet의 our_ref값
        #print(print_ques)
        #엑셀 sheet의 our_ref값 기준으로
        #Question모델의 id값을 찾아보자. <-- 모델의 전체 값을 filter기능으로 찾는것 테스트
        test_bb=Question.query.filter(Question.our_ref.ilike(print_ques)).all()
        print(test_bb)
        # => db내 해당 자료값 찾기 완료
        
        
        

최초 시작시 Question모델 데이터 개수 :  20
[<Question 1>]
[<Question 2>]
[<Question 3>]
[<Question 4>]
[<Question 5>]
[<Question 6>]
[<Question 7>]
[<Question 8>]
[<Question 9>]
[<Question 10>]
[<Question 11>]
[<Question 12>]
[<Question 13>]
[<Question 14>]
[<Question 15>]
[<Question 16>]
[<Question 17>]
[<Question 18>]
[<Question 19>]
[<Question 20>]


활용 엑셀 파일 내 our_ref 값 기준 db내 해당값 찾기를 완료 하였다.

이제 해당 내용을 수정할수 있도록 해보자.

In [14]:
a = 0
same_count = 0
test_bb=[]
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        # 엑셀 파일 내에 하나씩 읽어 오는 값을~~
        # 기존의 데이터와 비교해야 겠다.
        print_ques=ws.cell(row,1).value #<---- 엑셀 sheet의 our_ref값
        #print(print_ques)
        
        #엑셀 sheet의 our_ref값 기준으로
        #Question모델의 id값을 찾아보자. <-- 모델의 전체 값을 filter기능으로 찾는것 테스트
        test_bb=Question.query.filter(Question.our_ref.ilike(print_ques)).all()
        #print(test_bb) #<-- 기존 리스트로 되어있다.
        
        
        test_bb_00 = test_bb[0] #<-- 리스트내 값만 가져온다.
        #print(test_bb_00)
        # => db내 해당 자료값 찾기 완료
        
        
        test_bb_00.our_ref=ws.cell(row,1).value
        #question_input=test_bb_00(our_ref=ws.cell(row,1).value, pay_date=ws.cell(row,2).value, req_name=ws.cell(row,3).value, cus_name=ws.cell(row,4).value, date=ws.cell(row,5).value, number=ws.cell(row,6).value, q_date=ws.cell(row,7).value, q_d_date=ws.cell(row,8).value, d_d_date=ws.cell(row,9).value, d_d_num=ws.cell(row,10).value, p_name=ws.cell(row,11).value, charge_01=ws.cell(row,12).value, charge_02=ws.cell(row,13).value, p_charge=ws.cell(row,14).value, create_date=datetime.now())
        db.session.commit()
        same_count = same_count +1
        print('작업단계 : ', same_count)
        
        #성공 ===>>> 중요!!! 해당 내용은 our_ref값 기준으로 데이터를 찾고 수정하는것이다.
        # 본 예제에서는 our_ref값도 동일하게 변경하였지만 이후, our_ref값을 변경시키지 말고 다른 데이터를
        # 변경 시키는것을 해라

최초 시작시 Question모델 데이터 개수 :  20
작업단계 :  1
작업단계 :  2
작업단계 :  3
작업단계 :  4
작업단계 :  5
작업단계 :  6
작업단계 :  7
작업단계 :  8
작업단계 :  9
작업단계 :  10
작업단계 :  11
작업단계 :  12
작업단계 :  13
작업단계 :  14
작업단계 :  15
작업단계 :  16
작업단계 :  17
작업단계 :  18
작업단계 :  19
작업단계 :  20


▲▲▲▲ 위에까지 일단 원하는 기능 구현하였다.

▼▼▼▼ 아래서부터는 해당 기능 테스트와 개선을 진행한다. 2022-07-21

▼▼ 이번에는 "데이터 수정 적용 테스트" 를 실행해본다. 위에서는 our_ref값을 수정하는것으로 했지만

이제는 our_ref 기준으로 데이터 내 다른 열의 값을 수정하는 것을 목표로 하며 수정시 비어있는 cell 데이터가

어떻게 적용되는지(에러없이 되는지) 확인하는 단계이다.

In [12]:
a = 0
same_count = 0
test_bb=[]
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        # 엑셀 파일 내에 하나씩 읽어 오는 값을~~
        # 기존의 데이터와 비교해야 겠다.
        print_ques=ws.cell(row,1).value #<---- 엑셀 sheet의 our_ref값
        #print(print_ques)
        
        #엑셀 sheet의 our_ref값 기준으로
        #Question모델의 id값을 찾아보자. <-- 모델의 전체 값을 filter기능으로 찾는것 테스트
        test_bb=Question.query.filter(Question.our_ref.ilike(print_ques)).all()
        #print(test_bb) #<-- 기존 리스트로 되어있다.
        
        
        test_bb_00 = test_bb[0] #<-- 리스트내 값만 가져온다.
        #print(test_bb_00)
        # => db내 해당 자료값 찾기 완료
        
        """
        ▼▼▼ 이하 our_ref 값 기준으로 데이터네 열 값 수정 테스트 진행
        """
        #test_bb_00.our_ref=ws.cell(row,1).value #<--- 해당 부분
        test_bb_00.pay_date=ws.cell(row,2).value #<--- pay_date 기준 적용되게(해당부분 null값 존재)
        
        #question_input=test_bb_00(our_ref=ws.cell(row,1).value, pay_date=ws.cell(row,2).value, req_name=ws.cell(row,3).value, cus_name=ws.cell(row,4).value, date=ws.cell(row,5).value, number=ws.cell(row,6).value, q_date=ws.cell(row,7).value, q_d_date=ws.cell(row,8).value, d_d_date=ws.cell(row,9).value, d_d_num=ws.cell(row,10).value, p_name=ws.cell(row,11).value, charge_01=ws.cell(row,12).value, charge_02=ws.cell(row,13).value, p_charge=ws.cell(row,14).value, create_date=datetime.now())
        db.session.commit()
        same_count = same_count +1
        print('작업단계 : ', same_count)
        """
        ▲▲▲
        """
        
        #성공 ===>>> 중요!!! 해당 내용은 our_ref값 기준으로 데이터를 찾고 수정하는것이다.
        # 본 예제에서는 our_ref값도 동일하게 변경하였지만 이후, our_ref값을 변경시키지 말고 다른 데이터를
        # 변경 시키는것을 해라

최초 시작시 Question모델 데이터 개수 :  20
작업단계 :  1
작업단계 :  2
작업단계 :  3
작업단계 :  4
작업단계 :  5
작업단계 :  6
작업단계 :  7
작업단계 :  8
작업단계 :  9
작업단계 :  10
작업단계 :  11
작업단계 :  12
작업단계 :  13
작업단계 :  14
작업단계 :  15
작업단계 :  16
작업단계 :  17
작업단계 :  18
작업단계 :  19
작업단계 :  20


▲▲  "데이터 수정 적용 테스트"  성공

▼▼ 데이터 업로드 기능 수정하기_02_시작_220721

목표 : 엑셀 데이터 sheet에서 가져온 our_ref값이 데이터파일내 our_ref값중 해당되는게 있는지 참,거짓 형태로 출력하기(if문)

In [13]:
a = 0
same_count = 0
test_bb=[]
with app.app_context():
    print('최초 시작시 Question모델 데이터 개수 : ',Question.query.count())
    for row in range(1, ws.max_row+1):
        # 엑셀 파일 내에 하나씩 읽어 오는 값을~~
        # 기존의 데이터와 비교해야 겠다.
        print_ques=ws.cell(row,1).value #<---- 엑셀 sheet의 our_ref값
        print(print_ques)

최초 시작시 Question모델 데이터 개수 :  20
D211114
D211115
D211116
P212398
P212400
P212412
P211279-RE
P212405
P212418
P212430-DIV1
P212430-DIV2
P212419
P212420
P212421
P211363-PRO2
P212441
P209250-DIV1
P212439-PRO1
P212439-PRO2
P212434-PRO1


In [79]:
q = Question.query.get(2)

In [80]:
q

<Question 2>

In [56]:
q.our_ref='값 입력 테스트'

In [57]:
db.session.commit()

#-------------

#-------------

▼▼▼ 이하 예제 시작(pybo 예제)

In [1]:
from pybo.models import User
from pybo import db

In [2]:
from pybo.forms import UserCreateForm, UserLoginForm
from flask import Blueprint, url_for, render_template, flash, request, session, g
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import redirect

In [3]:
from flask import Flask
from flask_migrate import Migrate
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import MetaData
import config
import config


In [4]:
from pybo import db
import pybo

In [5]:
from openpyxl import load_workbook
from pybo.models import User
from datetime import datetime
from pybo import db, create_app
import pybo

▼▼▼ 예제중에 이게 있어야 했다.

In [6]:
from pybo import create_app
pybo = create_app()
pybo.app_context().push()

▲▲▲ 예제중에 이게 있어야 했다.

In [18]:
print(User.query.count())

14


In [12]:

app = Flask(__name__)
app.config.from_object(config)
    #app.config 환경 변수로 부르기 위해 app.config.from_object(config)코드를 추가함

In [44]:
with app.app_context():
    print(User.query.count())
    try:
        user = User(username='shin12350', password='shin_123450', email='shin50@naver.com')
        print(user)
        db.session.add(user)
        db.session.commit()
        print(User.query.count())
    except:
        db.session.rollback()
        raise
    finally:
        db.session.close()
    #db.create_all()

5
<User (transient 2597423975248)>
6


In [49]:
with app.app_context():
    print(User.query.count())
    try:
        user = User(username='shin12352', password='shin_123452', email='shin52@naver.com')
        print(user)
        db.session.add(user)
        db.session.commit()
        print(User.query.count())
        print('사용자 정보를 생성 하였습니다.')
    except:
        db.session.rollback()
        #raise
        print('사용중인 사용자 정보가 있습니다.')
        pass
    finally:
        db.session.close()
    #db.create_all()
print('종료')

7
<User (transient 2597424733200)>
8
사용자 정보를 생성 하였습니다.
종료


이제 연속해서 하는걸 해보자

원래 볼려고했던 최초 예제 참고

In [27]:
from openpyxl import load_workbook

In [32]:
wb = load_workbook(filename='db_test.xlsx')
ws = wb.active

In [33]:
ws

<Worksheet "Sheet1">

In [34]:
print(User.query.count())

20


In [35]:
a = 0
with app.app_context():
    print('최초 시작시 User모델 데이터 개수 : ',User.query.count())
    for row in range(1, ws.max_row+1):
        try:
            #user = User(username='shin12352', password='shin_123452', email='shin52@naver.com')
            user = User(username=ws.cell(row,1).value, password=ws.cell(row,2).value, email=ws.cell(row,3).value)
            print(user)
            db.session.add(user)
            db.session.commit()
            print('사용자 정보를 생성 하였습니다.')
        except:
            db.session.rollback()
            print('입력에 에러가 발생 하였습니다.')
            pass
        finally:
            db.session.close()
        print(a," 번째 작업을 종료 합니다.")
        a = a+1
        print('='*10)

print("최종 작업 이후 User모델 데이터 개수 : ", User.query.count())

최초 시작시 User모델 데이터 개수 :  20
<User (transient 2788504735120)>
입력에 에러가 발생 하였습니다.
0  번째 작업을 종료 합니다.
<User (transient 2788492368432)>
입력에 에러가 발생 하였습니다.
1  번째 작업을 종료 합니다.
<User (transient 2788504493984)>
입력에 에러가 발생 하였습니다.
2  번째 작업을 종료 합니다.
<User (transient 2788504735504)>
입력에 에러가 발생 하였습니다.
3  번째 작업을 종료 합니다.
<User (transient 2788504599952)>
입력에 에러가 발생 하였습니다.
4  번째 작업을 종료 합니다.
<User (transient 2788504694016)>
입력에 에러가 발생 하였습니다.
5  번째 작업을 종료 합니다.
<User (transient 2788504597072)>
입력에 에러가 발생 하였습니다.
6  번째 작업을 종료 합니다.
<User (transient 2788492356000)>
입력에 에러가 발생 하였습니다.
7  번째 작업을 종료 합니다.
<User (transient 2788504598368)>
입력에 에러가 발생 하였습니다.
8  번째 작업을 종료 합니다.
<User (transient 2788504598080)>
입력에 에러가 발생 하였습니다.
9  번째 작업을 종료 합니다.
<User (transient 2788504490192)>
입력에 에러가 발생 하였습니다.
10  번째 작업을 종료 합니다.
최종 작업 이후 User모델 데이터 개수 :  20


In [ ]:
a = 0
for row in range(1, ws.max_row+1):
    #print(a)
    vegan = ws.cell(row,1).value
    print(vegan)
    a = a+1
    print("종료시점 a : " ,a)
    print("="*10)

In [ ]:
with app.app_context():
    print(User.query.count())
    try:
        user = User(username='shin12352', password='shin_123452', email='shin52@naver.com')
        print(user)
        db.session.add(user)
        db.session.commit()
        print(User.query.count())
        print('사용자 정보를 생성 하였습니다.')
    except:
        db.session.rollback()
        #raise
        print('사용중인 사용자 정보가 있습니다.')
        pass
    finally:
        db.session.close()
    #db.create_all()
print('종료')